# Text Processing

In [2]:
import numpy as np
import pandas as pd
from pymystem3 import Mystem
import re
import transformers
import nltk
from tqdm import notebook
import torch
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from dateutil.parser import parse
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModel

## Данные

In [3]:
file_path = 'chd — 100.xlsx'

In [4]:
data = pd.read_excel(file_path)
data.head(1)

,admittion,department,discharge,sex,height,weight,BMI,BSA,birth,Операции (все в ИБ),...,Количество выб. из ОРИТ,К.дней в ОРИТ (всего),"ИВЛ, час. в ОРИТ (суммарно)",Инф. осложнения в ОРИТ,Назн. преп. в ОРИТ,target,Непосред. причина смерти,ЭхоКГ (Из Эпикр. вып.),ЭКГ (Из Эпикр. вып.),Назначения при выписке
0,2016-12-12,ehn,2017-01-10,m,76,9.7,111.27,0.46,02.01.2016,12.12.2016: (Откр./ИК) Перевязка ранее наложен...,...,2,20,252,01.01.2017: пневмония,"Адреналина г\хл 0,1% 1мл №5; Аксетин 750мг №10...",recovery,NaN,"ЭхоКГ ВПС (02.12.2016 14:37:22, врач Неталиева...",ЭКГ (02.12.2016 16:43:25)\nРитм сердца синусов...,


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 43 columns):
 #   Column                                           Non-Null Count  Dtype         
---  ------                                           --------------  -----         
 0   admittion                                        99 non-null     datetime64[ns]
 1   department                                       99 non-null     object        
 2   discharge                                        99 non-null     datetime64[ns]
 3   sex                                              99 non-null     object        
 4   height                                           99 non-null     int64         
 5   weight                                           99 non-null     float64       
 6   BMI                                              99 non-null     float64       
 7   BSA                                              99 non-null     float64       
 8   birth                                     

In [6]:
data['department'].value_counts()

ehn     46
onik    32
rhn     21
Name: department, dtype: int64

Для теста возьмем только один столбец - диагноз

In [44]:
diagnosis = data['Диагноз']
operations = data['Операции (все в ИБ)']

In [8]:
diagnosis[1]

'двойное отхождение аорты и легочной артерии от правого желудочка, подаортальный дефект межжелудочковой перегородки, комбинированный стеноз легочной артерии, умеренные сужения устьев правой и левой  легочных артерий; ОАП, НК 0-1 ст, артериальная гипоксемия, состояние после ТЛБВП 15.08.2016, клапана ЛА, с-м Дауна, с-м мышечной гипотонии, гипертензионно-гидроцефальный с-м; 18.01.2017 - ОПЕРАЦИЯ: радикальная коррекция двойного отхождения магистральных сосудов от правого желудочка с пластикой выводного отдела правого желудочка и ствола легочной артерии ксеноперикардиальной заплатой; перевязка открытого артериального протока; в условиях ИК, гипотеремии, НК 2а ст'

## Пример применения TF-IDF на столбце "Диагноз"

### Гипотетический способ отчистки конкретных элементов текста (например даты)

In [9]:
nDAY = r'(?:[0-3]?\d)'
nMNTH = r'(?:11|12|10|0?[1-9])' 
nYR = r'(?:(?:19|20)\d\d)'
nDELIM = r'(?:[\/\-\._])?'

In [10]:
re.sub(f'(?:{nDAY}{nDELIM}{nMNTH}{nDELIM}{nYR})', ' ', diagnosis[1])

'двойное отхождение аорты и легочной артерии от правого желудочка, подаортальный дефект межжелудочковой перегородки, комбинированный стеноз легочной артерии, умеренные сужения устьев правой и левой  легочных артерий; ОАП, НК 0-1 ст, артериальная гипоксемия, состояние после ТЛБВП  , клапана ЛА, с-м Дауна, с-м мышечной гипотонии, гипертензионно-гидроцефальный с-м;   - ОПЕРАЦИЯ: радикальная коррекция двойного отхождения магистральных сосудов от правого желудочка с пластикой выводного отдела правого желудочка и ствола легочной артерии ксеноперикардиальной заплатой; перевязка открытого артериального протока; в условиях ИК, гипотеремии, НК 2а ст'

In [58]:
operation_type = r'\(([a-zA-Zа-яА-ЯёЁ0-9_.-/ ]+)\)'

In [59]:
operations[1]

'18.01.2017: (Откр./ИК) Радикальная коррекция двойного отхождения магистральных сосудов от правого желудочка с пластикой выводного отдела правого желудочка и ствола легочной артерии ксеноперикардиальной заплатой; перевязка открытого артериального протока, в условиях ИК и гипотермии'

In [69]:
re.search(operation_type, operations[34])

<re.Match object; span=(12, 21), match='(Закрыт.)'>

### Лемментизация

In [11]:
m = Mystem()

# Леммантизирует текст
def lemmatize(text):
    return "".join(m.lemmatize(text))

In [12]:
lemmatize(diagnosis[1])

'двойной отхождение аорта и легочный артерия от правый желудочек, подаортальный дефект межжелудочковый перегородка, комбинированный стеноз легочный артерия, умеренный сужение устье правый и левый  легочный артерия; оап, НК 0-1 ст, артериальный гипоксемия, состояние после ТЛБВП 15.08.2016, клапан ЛА, с-м даун, с-м мышечный гипотония, гипертензионный-гидроцефальный с-м; 18.01.2017 - операция: радикальный коррекция двойной отхождение магистральный сосуд от правый желудочек с пластика выводной отдел правый желудочек и ствол легочный артерия ксеноперикардиальный заплата; перевязка открытый артериальный проток; в условие ик, гипотеремия, НК 2а ст\n'

### Отчистка текста

In [13]:
# Чистит текст от всего, кроме русских и английских букв
def clear_text(text):
    cleaned = re.sub(r'[^а-яА-Яa-zA-ZёЁ ]', ' ', text)
    cleaned = cleaned.split()
    return ' '.join(cleaned)

In [14]:
clear_text(lemmatize(diagnosis[1]))

'двойной отхождение аорта и легочный артерия от правый желудочек подаортальный дефект межжелудочковый перегородка комбинированный стеноз легочный артерия умеренный сужение устье правый и левый легочный артерия оап НК ст артериальный гипоксемия состояние после ТЛБВП клапан ЛА с м даун с м мышечный гипотония гипертензионный гидроцефальный с м операция радикальный коррекция двойной отхождение магистральный сосуд от правый желудочек с пластика выводной отдел правый желудочек и ствол легочный артерия ксеноперикардиальный заплата перевязка открытый артериальный проток в условие ик гипотеремия НК а ст'

### TF-IDF

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Стивен\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
stopwords = set(nltk_stopwords.words('russian'))

In [17]:
%%time

corpus = diagnosis.apply(lambda x: clear_text(lemmatize(x)))

CPU times: total: 93.8 ms
Wall time: 1min 18s


In [18]:
tf_idf = TfidfVectorizer(stop_words=stopwords).fit(corpus)
tf_idf_train = tf_idf.transform(corpus)

In [19]:
tf_idf_train.toarray().shape

(99, 481)

In [20]:
tf_idf.vocabulary_

{'атрезия': 40,
 'легочный': 203,
 'артерия': 34,
 'тип': 419,
 'перимембранозный': 286,
 'приточный': 331,
 'дефект': 120,
 'межжелудочковый': 212,
 'перегорока': 281,
 'некоммитированный': 236,
 'открывать': 269,
 'овальный': 254,
 'окно': 259,
 'стеноз': 405,
 'устье': 446,
 'левый': 201,
 'умеренный': 443,
 'гипоплазия': 94,
 'дискордантный': 125,
 'отхождение': 272,
 'аорта': 26,
 'правый': 326,
 'желудочек': 140,
 'состояние': 394,
 'операция': 261,
 'наложение': 228,
 'модифицированный': 221,
 'подключичный': 305,
 'анастомоз': 18,
 'blalock': 1,
 'справа': 399,
 'большой': 50,
 'аорто': 28,
 'коллатеральный': 171,
 'легкий': 202,
 'перевязка': 278,
 'ранее': 346,
 'налагать': 227,
 'реконструкция': 364,
 'путь': 340,
 'отток': 271,
 'кондуит': 179,
 'яремный': 480,
 'вена': 59,
 'бык': 54,
 'условие': 445,
 'ик': 149,
 'гипотермия': 96,
 'недостаточность': 235,
 'кровообращение': 190,
 'степень': 409,
 'двойной': 111,
 'подаортальный': 301,
 'перегородка': 279,
 'комбинированны

In [21]:
tf_idf_vocab = pd.DataFrame([[i, j] for i, j in tf_idf.vocabulary_.items()], columns=['word', 'index'])
tf_idf_vocab = tf_idf_vocab.set_index('index')
tf_idf_vocab = tf_idf_vocab.sort_index()
tf_idf_vocab.head(10)

,word
index,
0,alfieri
1,blalock
2,by
3,carbomedics
4,gore
5,ii
6,inversus
7,mastard
8,muller


## RuBert

In [23]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")

model = AutoModel.from_pretrained("cointegrated/rubert-tiny")

Downloading:   0%|          | 0.00/45.5M [00:00<?, ?B/s]

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
tokenized = corpus.apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

In [25]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
max_len

272

In [26]:
padded = np.array([i + [0] * (max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)
padded[0]

array([    2, 21642, 27846,  1512, 26335,  1172,  8034, 13013, 16140,
         776,  6116, 18807, 16890,   656, 13310, 14207,  1241,  1129,
        1026,  8034,  1395, 13650,  9164,   324, 15687,  3881,  3970,
        4823,  9041, 15998, 18807, 20836, 17781,   769,  2595,   864,
        1156, 25241,   733,   865, 24111,  1348, 12011,  7106, 13114,
         948,  3865,  3324,  1556,   331,  2242,   626, 26335, 15998,
       26335,  1172,  8034, 13013, 16140,   776, 11892, 10662,  3752,
       16615,  1231, 19917,   776, 26335, 15998, 26335,  1172,  8034,
       13013, 16140,   776, 20834, 28297, 16087,   733, 13718, 10389,
         312, 15526,   603,   733, 13329,  2841,  1712,  3970,  4823,
       16599, 22418,  1781, 20666,   548, 26541,   324, 16009,  8243,
        1835, 25241,  1464, 16785,  6981,  8034, 26335,  1172,  8034,
       20939,  9894, 20364,  1556,   705,    38,  9283,  6541, 22659,
        9989,   312, 15526,   721, 26335,  1172,  8034, 25415,  9823,
        3414,  7106,

In [33]:
batch_size = 9
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)
    
    embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/11 [00:00<?, ?it/s]

Stage 0
Stage 1
Stage 2
Stage 3
Stage 4
Stage 5
Stage 6
Stage 7
Stage 8
Stage 9
Stage 10


In [34]:
features = np.concatenate(embeddings)

In [35]:
features.shape

(99, 312)

In [36]:
features

array([[ 0.19064732,  0.10095225,  0.7029128 , ..., -0.2642672 ,
        -0.44361597, -0.62070465],
       [ 0.34114507,  0.25692418,  0.8168308 , ..., -0.12020289,
        -0.48846975, -0.68282175],
       [-0.12180065, -0.02276124,  0.6054246 , ..., -0.0247761 ,
        -0.2868384 , -0.5125958 ],
       ...,
       [ 0.16963647, -0.15543881,  0.6822833 , ..., -0.13596621,
        -0.45828795, -0.6662678 ],
       [-0.06635535,  0.22518474,  0.29201087, ...,  0.26825568,
         0.09903832, -0.7719879 ],
       [ 0.3412424 ,  0.37697873,  0.8917906 , ..., -0.1149576 ,
        -0.56047875, -0.6824301 ]], dtype=float32)

Получилось 120000 признаков. Векторизация 100 примеров проходила 40 минут. **Надо использовать модель с меньшим словарем, желательно из медицинских терминов.**

## Тест модели

In [37]:
X, y = features, data['department']

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [39]:
model = CatBoostClassifier(loss_function='MultiClass', verbose=100)
model = model.fit(X_train, y_train, eval_set=(X_test, y_test))

Learning rate set to 0.102631
0:	learn: 1.0715365	test: 1.0885036	best: 1.0885036 (0)	total: 183ms	remaining: 3m 2s
100:	learn: 0.1190856	test: 0.7269265	best: 0.7251123 (93)	total: 2.48s	remaining: 22.1s
200:	learn: 0.0441446	test: 0.6572254	best: 0.6566778 (199)	total: 4.78s	remaining: 19s
300:	learn: 0.0256447	test: 0.6540424	best: 0.6540424 (300)	total: 7.1s	remaining: 16.5s
400:	learn: 0.0176745	test: 0.6538741	best: 0.6530707 (396)	total: 9.37s	remaining: 14s
500:	learn: 0.0134747	test: 0.6528694	best: 0.6513977 (465)	total: 11.6s	remaining: 11.6s
600:	learn: 0.0109132	test: 0.6555131	best: 0.6513977 (465)	total: 13.9s	remaining: 9.23s
700:	learn: 0.0091992	test: 0.6559474	best: 0.6513977 (465)	total: 16.2s	remaining: 6.9s
800:	learn: 0.0078884	test: 0.6571646	best: 0.6513977 (465)	total: 18.4s	remaining: 4.58s
900:	learn: 0.0069314	test: 0.6589907	best: 0.6513977 (465)	total: 20.7s	remaining: 2.28s
999:	learn: 0.0061457	test: 0.6610390	best: 0.6513977 (465)	total: 23s	remaining:

In [39]:
tf_idf_vocab['importance'] = model.get_feature_importance()
tf_idf_vocab_sorted = tf_idf_vocab.sort_values(by='importance', ascending=False)
tf_idf_vocab_sorted.head(10)

,word,importance
index,,
244,нк,43.594357
409,степень,22.071899
400,ст,17.487092
316,последствие,1.341840
261,операция,1.113037
120,дефект,0.788361
416,тетрада,0.664262
437,трикуспидальный,0.654644
26,аорта,0.594327


In [42]:
tf_idf_vocab_sorted.head(15)['importance'].sum()

90.78744575284654

15 слов из топа по важности имеют самое большое врияние. Для оптимизации можно оставить только фичи из этого топа.

In [40]:
print(classification_report(y_test, model.predict(X_test)))

              precision    recall  f1-score   support

         ehn       1.00      0.92      0.96        12
        onik       0.50      1.00      0.67         2
         rhn       0.80      0.67      0.73         6

    accuracy                           0.85        20
   macro avg       0.77      0.86      0.78        20
weighted avg       0.89      0.85      0.86        20



In [41]:
cat_score = cross_val_score(model, X, y)
cat_score

Learning rate set to 0.069519
0:	learn: 1.0780948	total: 29.2ms	remaining: 29.2s
100:	learn: 0.2254463	total: 2.38s	remaining: 21.2s
200:	learn: 0.0776208	total: 4.71s	remaining: 18.7s
300:	learn: 0.0437027	total: 7s	remaining: 16.2s
400:	learn: 0.0298915	total: 9.28s	remaining: 13.9s
500:	learn: 0.0225179	total: 11.6s	remaining: 11.5s
600:	learn: 0.0180843	total: 14s	remaining: 9.29s
700:	learn: 0.0150530	total: 16.4s	remaining: 6.98s
800:	learn: 0.0128716	total: 18.8s	remaining: 4.66s
900:	learn: 0.0111999	total: 21.1s	remaining: 2.32s
999:	learn: 0.0099785	total: 23.4s	remaining: 0us
Learning rate set to 0.069519
0:	learn: 1.0769836	total: 28.8ms	remaining: 28.7s
100:	learn: 0.2286972	total: 2.28s	remaining: 20.3s
200:	learn: 0.0808205	total: 4.59s	remaining: 18.3s
300:	learn: 0.0461829	total: 6.88s	remaining: 16s
400:	learn: 0.0310226	total: 9.15s	remaining: 13.7s
500:	learn: 0.0232749	total: 11.4s	remaining: 11.4s
600:	learn: 0.0184880	total: 14.2s	remaining: 9.45s
700:	learn: 0.0

array([0.8       , 0.75      , 0.8       , 0.7       , 0.78947368])

In [36]:
cat_score.mean()

0.9800000000000001

In [29]:
dummy_model = DummyClassifier().fit(X_train, y_train)
print(classification_report(y_test, dummy_model.predict(X_test)))

              precision    recall  f1-score   support

         ehn       0.75      1.00      0.86        15
        onik       0.00      0.00      0.00         4
         rhn       0.00      0.00      0.00         1

    accuracy                           0.75        20
   macro avg       0.25      0.33      0.29        20
weighted avg       0.56      0.75      0.64        20



C:\anaconda\envs\practicum\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\anaconda\envs\practicum\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\anaconda\envs\practicum\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [48]:
cross_val_score(DummyClassifier(), X, y)

array([0.5       , 0.45      , 0.45      , 0.45      , 0.47368421])

In [47]:
cross_val_score(DecisionTreeClassifier(), X, y)

array([1.  , 1.  , 0.95, 1.  , 1.  ])